
> **Atividade** (Sendo aprimorada):
> 1. Adapte o código acima para fazer com que a função `solve_triangular_inferior()` também retorno o número de operações (somas, produtos, divisões e subtrações numéricas) que foram realizadas. (**Dica**: A resposta é $n^2$; tente deduzir este valor formalmente).
> 2. Adapte o código acima para fazer o processo análogo (também contando operações), mas com matrizes triangulares _inferiores_.
> 3. Adapte o algoritmo de Crout&ndash;Dolittle, para fazer com que ele também retorne o número de operações realizadas.
> 4. Utilize as funções criadas no passo anterior para criar uma função que recebe uma matriz $A$, um inteiro positivo $n$ e um vetor $b$ e
>     - Decompõe a matriz $A$ como $A=LU$.
>     - Resolve o sistema linear $A^n x = b$, com aplicações sucessivas das funções criadas nos passos 1. e 2.
>     - Conta o número de operações.
> 5. Adapte o algoritmo de resolução de sistemas lineares da aula 9 para que ele conte o número de operações realizadas.
> 6. Adapte o algoritmo abaixo, que realiza o produto de duas matrizes $A$ e $B$, para que ele também conte o número de operações realizadas.
> 7. Crie uma matriz aleatória $A$ de ordem $4\times 4$ e um vetor $b$ de tamanho $4$, e resolva o sistema $A^{20}x = b$ por dois modos:
>     - Utilizando o método do passo 4;
>     - Calculando o produto $A^{20}=A\cdot A\cdots A$ com o algoritmo do passo 6, e resolvendo o sistema $(A^n)x = b$ diretamente com o algoritmo do passo 5.
>
>     Compare os números de operações e os resultados.


### 1 -Adaptar o código da superior pra contar as operações

In [29]:
import numpy as np

def solve_triangular_superior(U, b, op):

    n = U.shape[0]         
    x = b.copy().reshape(n)

    # Vai linha-a-linha, de ***baixo para cima***
                        
    for i in range(n-1,-1,-1):
        x[i] /= U[i,i]     # Normaliza a i-ésima linha
        op+=1              # 1 divisão
        for j in range(i-1,-1,-1):
            x[j] -= U[j,i]*x[i]     # Pivoteia a i-ésima coluna, utilizando a entrada diagonal como pivô
            op+=2                   # subtraçao e divisão

    return x,op


U = 20*np.random.rand(4,4) -10 

for i in range(4):
    for j in range(i):
        U[i,j]=0.0        # Aniquila as entradas abaixo da diagonal principal para deixar triangular superior
        
        
b = 20*np.random.rand(4) - 10 
operacoesU=0
x,operacoesU = solve_triangular_superior(U,b,operacoesU) # Solução pretendida do sistema Ux = b

print(f"\nA matriz U é dada por\n{U}")
print(f"\nO vetor b é dada por\n{b}")
print(f"\nO vetor x é dada por\n{x}")
print(f"\nO vetor Ux é dada por\n{U@x}")
print(f"\nO numero de operações de uma matriz 4x4 é: \n{operacoesU}")





A matriz U é dada por
[[-8.30717233 -1.63706639  1.21252039  2.57977911]
 [ 0.          4.99850299 -4.24153279  4.60898428]
 [ 0.          0.         -8.45316574 -9.14839781]
 [ 0.          0.          0.         -5.21941826]]

O vetor b é dada por
[-9.91592452  9.49862714  6.33467021 -8.51716177]

O vetor x é dada por
[ 1.67593496 -1.73884742 -2.51541589  1.63182204]

O vetor Ux é dada por
[-9.91592452  9.49862714  6.33467021 -8.51716177]

O numero de operações de uma matriz 4x4 é: 
16


### 2 -Criando a triangular inferior

In [30]:
def solve_triangular_inferior(L, B,op):
    """Resolve um sistema triangular inferior do tipo Ly = B"""
    n = L.shape[0]        
    X = B.copy().reshape(n)
    
    for i in range(n):
        for j in range(i):
            X[i] -= L[i,j] * X[j]    # Pivoteia a i-ésima coluna, utilizando a entrada diagonal como pivô
            op+=2                    #1 soma, 1 multiplicação
        X[i] /= L[i,i]               # Normaliza a i-ésima linha
        op+=1
    return X, op
          
L = 20*np.random.rand(4,4) -10 
for i in range(4):
    for j in range(i):
        L[j,i]=0.0        # Aniquila as entradas acima da da diagonal principal para deixar triangular inferior
        
b = 20*np.random.rand(4) - 10 
operacoesL = 0
y,operacoesL = solve_triangular_inferior(L,b,operacoesL) # Solução pretendida do sistema Ly = x

print(f"\nA matriz L é dada por\n{L}")
print(f"\nO vetor b é dada por\n{b}")
print(f"\nO vetor y é dada por\n{y}")
print(f"\nO vetor Ly é dada por\n{L@y}")
print(f"\nO numero de operacões para uma matriz 4x4 é:\n{operacoesL}")



A matriz L é dada por
[[-9.1454265   0.          0.          0.        ]
 [-2.26133259 -1.80824433  0.          0.        ]
 [ 6.44570384  9.3894413  -6.07031442  0.        ]
 [ 6.83582278  9.30575835  2.32774765  4.44243486]]

O vetor b é dada por
[ 3.21872468  6.78507542 -1.64833087 -5.72970352]

O vetor y é dada por
[-0.35194911 -3.31216382 -5.22536304  8.92791501]

O vetor Ly é dada por
[ 3.21872468  6.78507542 -1.64833087 -5.72970352]

O numero de operacões para uma matriz 4x4 é:
16


### Questão 3
#### Contando as operaçoes de Doliru 
`retornando numero de operações diferente do esperado (41)`

In [31]:
def crout_dolittle(A, opD):
    ''' Decomposição LU de A pelo algoritmo de Crout'''

    m,n = np.shape(A) # pega as dimenções

    L = np.zeros((n,n)) # cria matriz de zeros
    U = np.zeros((n,n))

    L[0,0] = 1          # Escolha de Dolittle
    U[0,0] = A[0,0]

    for j in range(1,n):
        U[0,j] = A[0,j]/L[0,0]      # Determina a primeira linha de U
        L[j,0] = A[j,0]/U[0,0]      # Determina a primeira coluna de L
        opD+=2
    for i in range(1,n):
        L[i,i]=1        # Escolha de Dolittle
        U[i,i] = A[i,i] - sum([L[i,k]*U[k,i] for k in range(i)])
        opD += 2*n
        for j in range(i+1,n):
            U[i,j] = (A[i,j] - sum([L[i,k]*U[k,j] for k in range(i)]))/L[i,i]       # Determina a i-ésima linha de U
            opD+=2*i-1
            L[j,i] = (A[j][i] - sum([L[j,k]*U[k,i] for k in range(i)]))/U[i,i]      # Determina a i-ésima coluna de L
            opD+=2*i-1

    return L , U, opD


opD=0
A = 20*np.random.rand(4,4) - 10.0       # Matriz aleatória com entradas entre -10 e 10

L , U, opD = crout_dolittle(A,opD)               # Decomposição LU de $A$

print(f"A matriz A é dada por\n{A}")
print(f"\nA matriz L é dada por\n{L}")
print(f"\nA matriz U é dada por\n{U}")
print(f"\n\nO produto LU é dado por\n{L@U}")
print(f"O numero de operaçoes foi: \n{opD}")

A matriz A é dada por
[[-5.07524149 -9.99290794  6.59079772 -8.11840193]
 [ 1.73838993 -1.37186769  9.4641021   2.4555381 ]
 [-2.51994603 -7.25918209 -7.56094528  5.28122441]
 [ 0.7922483   6.06088857  1.70442152 -7.75694319]]

A matriz L é dada por
[[ 1.          0.          0.          0.        ]
 [-0.34252359  1.          0.          0.        ]
 [ 0.49651746  0.47918349  1.          0.        ]
 [-0.15610061 -0.93874769 -0.83505895  1.        ]]

A matriz U é dada por
[[ -5.07524149  -9.99290794   6.59079772  -8.11840193]
 [  0.          -4.79467439  11.7216058   -0.32520607]
 [  0.           0.         -16.45019138   9.46798612]
 [  0.           0.           0.          -1.42319062]]


O produto LU é dado por
[[-5.07524149 -9.99290794  6.59079772 -8.11840193]
 [ 1.73838993 -1.37186769  9.4641021   2.4555381 ]
 [-2.51994603 -7.25918209 -7.56094528  5.28122441]
 [ 0.7922483   6.06088857  1.70442152 -7.75694319]]
O numero de operaçoes foi: 
40


### Questão 04

In [42]:
def quest04(A,B,n,op):
    L, U, op = crout_dolittle(A,op)
    x=B
    for i in range (n):
        y,op = solve_triangular_inferior(L,x,op)
        x,op = solve_triangular_superior(U,y,op)
    return x, op

## Questão 05

In [1]:
import numpy as np

def pivot_partial(x):
    return np.argmax(abs(x))

def ref(A, tol=None, pivot=pivot_partial, verbose=False):
    
    #Faz cópias
    T = np.array(A).astype(float)
    posicao_pivos = []
    #Grava o tamanho
    n_linhas, n_colunas = np.shape(T)


    if tol == None:
        tol = 2**(-52) * np.max(abs(T)) * max(n_linhas, n_colunas)
        #Optanto por não contar esses calculos para saber apenas o numero de operações da f e não de requisitos de comparação para tol
    

    #Vê quantos pivôs já foram achados
    n_pivos = 0

    #Linhas na qual trabalharemos
    j = 0

    if verbose:
        print("Vamos escalonar parcialmente a matriz")
        print(T)

    num_op = 0
    
    while (j < n_colunas and n_pivos < n_linhas):
        if verbose:
            print("=====")
            print(f"Vamos pivotear a coluna {j}.")

        #Encontra o pivô
        p = pivot(T[n_pivos:, j]) + n_pivos
        if abs(T[p, j]) > tol:
            #Encontramos um pivô.
            #Troca linhas caso necessário
            if p != n_pivos:

                for k in range(j, n_colunas):
                    temp = T[p, k]
                    T[p, k] = T[n_pivos, k]
                    T[n_pivos, k] = temp

                #end for
                if verbose:
                    print(
                        f"Precisamos trocar a linha {n_pivos} com a linha {p}."
                    )
                    print(T)
                #end if

                p = n_pivos
            #end if

            #Pivoteia abaixo
            for k in range(p + 1, n_linhas):
                if abs(T[k, j]) > tol:
                    multiplicador = T[k, j] / T[p, j]
                    num_op += 1
                    T[k, j + 1:] = T[k, j + 1:] - multiplicador * T[p, j + 1:]
                    num_op += n_colunas - 1 - j
                    T[k, j] = 0
                #end if
            #end for
            if verbose:
                print("Aniquila as entradas abaixo:")
                print(T)
            #end if

            #Conta o pivô a mais
            n_pivos += 1
            posicao_pivos.append(j)
        else:
            if verbose:
                print(f"A coluna {j} nao tem pivo.")
        #end if
        #passa pra próxima coluna
        j += 1
    #end while

    if verbose:
        print(f"Numero de operacoes: {num_op}")
    #end if-else
    return [T, posicao_pivos,num_op]


#end def


def retrossub(A, pospiv=[], tol=None, verbose=False):
    R = np.array(A).astype(float)
    m, n = np.shape(R)
    num_op=0
    if tol == None:
        tol = 2**(-30) * np.max(abs(R))
    if pospiv == []:
        j = 0
        while (i < m):
            while abs(R[i, j]) < tol and j < n:
                j += 1
            if j == n:
                break
            else:
                pospiv.append(j)
                i += 1
    numero_de_pivos = len(pospiv)
    for i in range(numero_de_pivos - 1, -1, -1):
        if abs(R[i, pospiv[i]] - 1) > tol:
            R[i, pospiv[i] + 1:] /= R[i, pospiv[i]]
            num_op+=1
            R[i, pospiv[i]] = 1
        for k in range(i - 1, -1, -1):
            R[k, pospiv[i] + 1:] -= R[k, pospiv[i]] * R[i, pospiv[i] + 1:]
            num_op+=2
            R[k, pospiv[i]] = 0
    return [R, pospiv,num_op]


def rref(A, tol=None, pivot=pivot_partial, verbose=False):
    cont=0
    T = ref(A, tol=tol, pivot=pivot, verbose=verbose)
    cont+=T[2]
    resultado=retrossub(T[0], pospiv=T[1], tol=tol, verbose=verbose)
    return resultado

A = np.random.randint(0,5, size=(4,4)) 
print(rref(A,None,pivot_partial,1))


#end def

Vamos escalonar parcialmente a matriz
[[4. 0. 0. 4.]
 [3. 4. 1. 4.]
 [3. 3. 0. 3.]
 [0. 2. 2. 0.]]
=====
Vamos pivotear a coluna 0.
Aniquila as entradas abaixo:
[[4. 0. 0. 4.]
 [0. 4. 1. 1.]
 [0. 3. 0. 0.]
 [0. 2. 2. 0.]]
=====
Vamos pivotear a coluna 1.
Aniquila as entradas abaixo:
[[ 4.    0.    0.    4.  ]
 [ 0.    4.    1.    1.  ]
 [ 0.    0.   -0.75 -0.75]
 [ 0.    0.    1.5  -0.5 ]]
=====
Vamos pivotear a coluna 2.
Precisamos trocar a linha 2 com a linha 3.
[[ 4.    0.    0.    4.  ]
 [ 0.    4.    1.    1.  ]
 [ 0.    0.    1.5  -0.5 ]
 [ 0.    0.   -0.75 -0.75]]
Aniquila as entradas abaixo:
[[ 4.   0.   0.   4. ]
 [ 0.   4.   1.   1. ]
 [ 0.   0.   1.5 -0.5]
 [ 0.   0.   0.  -1. ]]
=====
Vamos pivotear a coluna 3.
Aniquila as entradas abaixo:
[[ 4.   0.   0.   4. ]
 [ 0.   4.   1.   1. ]
 [ 0.   0.   1.5 -0.5]
 [ 0.   0.   0.  -1. ]]
Numero de operacoes: 16
[array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]]), [0, 1, 2, 3], 16]


### 6 - Produto de matrizes
#### Teste e adaptação para contar operações


In [38]:
def matmul(A,B,op):
    
    n=A.shape[0]
    
    C = np.zeros((n,n))
    
    for i in range(n):
        for j in range(n):
            for k in range(n):
                C[i,j] += A[i,k]*B[k,j]
                op+=2
    return C, op

A = np.random.randint(0,5, size=(3,3)) 
B = np.random.randint(0,5, size=(3,3)) 
opProduto = 0
C, opProduto= matmul(A,B,opProduto)

print(f"A matriz A é dada por\n{A}")
print(f"\nA matriz L é dada por\n{B}")
print(f"\nA matriz C é dada por\n{C}")
print(f"\nO numero de operações é: \n{opProduto}")

A matriz A é dada por
[[1 3 4]
 [2 0 4]
 [2 2 1]]

A matriz L é dada por
[[2 0 4]
 [4 0 1]
 [4 1 3]]

A matriz C é dada por
[[30.  4. 19.]
 [20.  4. 20.]
 [16.  1. 13.]]

O numero de operações é: 
54


### Questão 7
#### Multiplicalções sucessivas pra (A^20)x = b



In [89]:
def quest07(Aa,Bb,op):
    A = np.copy(Aa)

    for i in range(19):            #laço n vezes  (0 a n-1)
        A,op = matmul(A,Aa,op)     #Multiplica A por A 20 vezes
    
    L,U, op= crout_dolittle(A,op) #Decompõe já contando as operações
    # RESOLVER O SISTEMA Ax = b
    #                    LUx = b
    #                   ==========
    #                  / Ly = b
    #                  \ Ux = y
    #                   =========
    
    y,op = solve_triangular_inferior(L,Bb,op)  #Resolve e volta y tendo L e B da função Ly = b
    x,op =solve_triangular_superior(U,y,op)   #Resolve e volta x tendo U e y da função Ux = y
    return x, op

In [97]:
# Criação de matrizes para o teste e comparação das duas questões
A = 20*np.random.rand(4,4) -10
B = 20*np.random.rand(4) - 100000
Aa = np.copy(A)        

Bb = np.copy(B)

op4 = 0
op7 = 0

X, op4 = quest04(A,B,20,op4)
Y, op7 = quest07(Aa,Bb,op7)




print(f"A matriz A é dada por\n{A}")
print(f"\nA matriz B é dada por\n{B}")
print("==================================")
print(f"Resultado da questão 4: \n{X}")
print(f"Resultado da questão 7: \n{Y}")
print("==================================")
print(f"Numero de Operações da Questão 4: {op4}")
print(f"Numero de Operações da Questão 7: {op7}")

A matriz A é dada por
[[-2.42421668e+00  6.79160631e+00  8.41598966e+00  3.92146687e+00]
 [ 4.70307382e+00  5.92295236e-01 -4.34944931e+00 -7.16573234e+00]
 [ 7.13333630e-03  9.19374047e+00 -2.74778722e+00  3.79132312e+00]
 [ 7.94436049e+00  1.49267119e+00 -2.38011029e-01  9.19132099e+00]]

A matriz B é dada por
[-99983.66946807 -99983.91284476 -99986.374424   -99999.56507169]
Resultado da questão 4: 
[-4.04751026e-16 -3.12600058e-15  4.13791010e-16  6.79622574e-15]
Resultado da questão 7: 
[-4.04751026e-16 -3.12600058e-15  4.13791010e-16  6.79622574e-15]
Numero de Operações da Questão 4: 680
Numero de Operações da Questão 7: 2504


Como A é uma matriz elevada a 20, aumentamos o intervalo de B para que o resultado das operações não dar Nan ou numeros muito proximos de 0

Contudo, é possível observar que o uso de do método `Crout Dolittle` em cadeia possui menas operações do que a multiplicação sucessiva matriz A